In [1]:
import cv2
import os
import glob
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [2]:
char_w = 30
char_h = 60

In [3]:
pathTest = '.\\data\\2\\1.jpg'

img = cv2.imread(pathTest, 0)
  
# Displaying the image
cv2.imshow('image', img)
cv2.waitKey(0);
cv2.destroyAllWindows(); 
cv2.waitKey(1)

print(type(img.shape))
print(img.shape)
print(type(np.array(img)))




<class 'tuple'>
(60, 30)
<class 'numpy.ndarray'>


In [4]:
def get_char_data(path):
    
    char_list = []
    label_list = []

    # label digit
    for num_iter in range(10):
        
        # print(num_iter)

        for img_path in glob.iglob(path + str(num_iter) + '/*.jpg'):
            # print(img_path)
            # flags = 0 to take gray scale img
            img = cv2.imread(img_path, 0)
            # convert tuple to np.arr
            img = np.array(img)
            img = img.reshape( -1, char_h * char_w)

            # print(img.shape)

            char_list.append(img)
            label_list.append( int(num_iter)  )

    # label alphabet

    for alpha_iter in range(65, 91):
        # print(alpha_iter)
        for img_path in glob.iglob(path + str(alpha_iter) + '/*.jpg'):
            # print(img_path)

            img = cv2.imread(img_path, 0)
            img = np.array(img)
            img = img.reshape(-1, char_h * char_w)

            # print(img.shape)

            char_list.append(img)
            label_list.append( int(alpha_iter) )

    return char_list, label_list 

In [9]:
# data_path = 'data/'
data_path = 'data2/'
char_list, label_list = get_char_data(data_path)


char_list = np.array(char_list, dtype = np.float32)
label_list = np.array(label_list)

char_list = char_list.reshape(-1, char_h * char_w)
label_list = label_list.reshape(-1, 1)

# print(char_list.shape)
# print(label_list.shape)

svm_model = cv2.ml.SVM_create()
svm_model.setType(cv2.ml.SVM_C_SVC)

svm_model.setKernel(cv2.ml.SVM_RBF)
svm_model.setTermCriteria( (cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6) )

svm_model.train(char_list, cv2.ml.ROW_SAMPLE, label_list)



# svm_model.save('svm_INTER.xml')
svm_model.save('svm_RBF.xml')

# Accuracy, Precision, Recall, F-Score,... for 2 Class:
<img height="180em" src = "https://raw.githubusercontent.com/thangnch/photos/master/Screen%20Shot%202020-06-16%20at%2014.00.30.png">

## Accuracy, Precision, Recall, F-Score,.. for Multi Class:

- [PDF](http://rali.iro.umontreal.ca/rali/sites/default/files/publis/SokolovaLapalme-JIPM09.pdf)

In [6]:
def get_presision_and_recall(model_path, data_pred_path):
    
    svm_model = cv2.ml.SVM_load(model_path)
    char_list, label_list = get_char_data(data_pred_path)
    

    total_Precision = 0
    total_Recall = 0
    
    unique_Label = set(label_list)

    step = 0
    total_step = len(unique_Label) * len(char_list)
    # print(label_list)
    # print(unique_Label)

    for iter in  unique_Label :
        true_Positive = 0
        false_Positive = 0
        true_Negative = 0
        false_Negative = 0

        single_Precision = 0
        single_Recall = 0

        for j in range( len(char_list)):
            
            # print(str(step) + "/" + str(total_step) )
            step += 1

            char_list[j] = np.array(char_list[j], dtype = np.float32)
            # print(char_list[j] )
            # print(char_list[j].shape)


            pred = svm_model.predict( char_list[j] )[1]
            pred = pred[0][0]
            if( pred == iter and iter  == label_list[j] ):
                true_Positive +=1
            elif( pred != iter and iter != label_list[j] ):
                true_Negative +=1
            elif( pred == iter and iter != label_list[j]):
                false_Positive +=1
            else:
                false_Negative +=1
            
        single_Precision = true_Positive / (true_Positive + false_Positive)
        single_Recall = true_Positive / (true_Positive + false_Negative)

        total_Precision += single_Precision
        total_Recall += single_Recall
    
    return total_Precision / len(unique_Label), total_Recall / len(unique_Label) 
        

        
def get_Average_Accuracy_and_Error_rate(model_path, data_pred_path):

    svm_model = cv2.ml.SVM_load(model_path)
    char_list, label_list = get_char_data(data_pred_path)

    
    total_Binary_Accuracy = 0
    total_Error_rate = 0
    
    unique_Label = set(label_list)

    # print(label_list)
    # print(unique_Label)
    step = 0
    total_step = len(unique_Label) * len(char_list)
    
    for iter in  unique_Label :
        true_Positive = 0
        true_Negative = 0
        Binary_Accuracy = 0
        single_Error_rate = 0

        for j in range( len(char_list)):
            
            # print(str(step) + "/" + str(total_step) )
            step += 1
            
            char_list[j] = np.array(char_list[j], dtype = np.float32)
            # print(char_list[j] )
            # print(char_list[j].shape)


            pred = svm_model.predict( char_list[j] )[1]
            pred = pred[0][0]
            if( pred == iter and iter  == label_list[j] ):
                true_Positive +=1
            elif( pred != iter and iter != label_list[j] ):
                true_Negative +=1
            else:
                single_Error_rate += 1
            
        Binary_Accuracy = (true_Positive + true_Negative) / len(char_list)
        total_Binary_Accuracy += Binary_Accuracy

        single_Error_rate /= len(char_list)
        total_Error_rate += single_Error_rate

    return total_Binary_Accuracy / len( unique_Label ), total_Error_rate / len(unique_Label)            

def get_F_Score(precision, recall, beta):
    
    return (beta * beta + 1) * (precision * recall) / (beta * beta * precision + recall)


In [13]:
model_path = 'svm_RBF.xml'
data_pred_path = 'data/' 
a_e = get_Average_Accuracy_and_Error_rate(model_path, data_pred_path)
print("Accuracy: "   + str( a_e[0] ) )
print("Error rate: " + str( a_e[1] ) )

# p_rc = get_presision_and_recall(model_path, data_pred_path)
# print("Precision: "  + str( p_rc[0] ) )
# print("Recall: "     + str( p_rc[1] ) )

# beta = 1
# fs = get_F_Score(p_rc[0], p_rc[1],beta)
# print(f"F-{beta} Score: "  + str( fs )      )


# os.chdir('E:\Sem5\AIP391\src\Character_detection_with_SVM')
# fh = open('Confusion_Matrix.txt', "w+")
# fh = open('Confusion_Matrix.txt', "a+")

# fh.write("Accuracy: "   + str( a_e[0] ))
# fh.write("Error rate: " + str( a_e[1] ) )
# fh.write("Precision: "  + str( p_rc[0] ) )
# fh.write("Recall: "     + str( p_rc[1] ) )
# fh.write("F-"+str(beta)+": "  + str( fs )      )

Accuracy: 0.9860738823674399
Error rate: 0.013926117632560003


In [8]:
# test predict
import cv2
import numpy as np

SVM_mol = cv2.ml.SVM_load('svm2.xml')
# print(SVM_mol.isTrained)
# print(SVM_mol.isClassifier)


img_test_path = r'test_img/3.jpg'

img_test = cv2.imread(img_test_path, 0)

cv2.imshow('GrayScale', img_test)
cv2.waitKey();

img_test = cv2.resize(img_test, dsize = ( char_w, char_h) ) 

img_test = np.array(img_test, dtype = np.float32)
print(img_test)
print(img_test.shape)

cv2.imshow('After resize', img_test)
cv2.waitKey();



# img_test = 
# print(img_test.shape)

img_test = img_test.reshape(-1, char_w * char_h)

print(img_test)
print(img_test.shape)


print(SVM_mol.predict(img_test))
pred = SVM_mol.predict(img_test)[1]

pred = pred[0][0]

print('predict value: ',end= '')
if pred < 10:
    print(pred)
else: 
    print(chr(pred))






[[  1.   0.   1. ...   0.   1.   1.]
 [  1.   1. 169. ... 170.   0.   1.]
 [  1. 102. 255. ... 254.   0.   0.]
 ...
 [  1. 203. 255. ... 255. 255.  67.]
 [  1.   1. 169. ... 170.   1.   1.]
 [  0.   1.   0. ...   1.   1.   1.]]
(60, 30)
[[1. 0. 1. ... 1. 1. 1.]]
(1, 1800)
(0.0, array([[88.]], dtype=float32))
predict value: X
